<h1 align="center">Hyperparamter Tunning</h1>

## Integrantes

- Josué Say
- Andre Jo

## Repositorio

- [Enlace a GitHub](https://github.com/JosueSay/labs_dl/tree/main/lab2)

In [40]:
# %pip install -r requirements.txt

## Carga del Dataset

### Librerías y constantes

In [41]:
import os
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torch.utils.data import random_split

BASE_DIR = "./data/"
BATCH_SIZE = 64
FACTOR_VALIDATION_SET = 0.9
os.makedirs(BASE_DIR, exist_ok=True)

### Preprocesamiento

In [42]:
def getParamsTransform(bs):
    
    # El dataset de pytorch obtiene la data en formato PIL (Python Image Library) por lo que debemos pasarlo a 
    # tensores y que la red neuronal sea más fácil en procesar y luego transformamos sus datos de imágenes en rango 
    # `[0-255]` a `[0-1]`, esto sería una transformación base porque debemos al transformar datos debemos de actualizar 
    # la distribución y eso lo hacemos sabiendo la `mean` y `std`.
    base_transform = v2.Compose([
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True)
    ])
     
    # Descargamos la data para train y test en caso que no este en `BASE_DIR` lo descargará sino solo lo leera de la carpeta `data`. Se aplica la transformación base.
    train_data = datasets.MNIST(root=BASE_DIR, train=True, download=True, transform=base_transform)
    
    # Cargamos los datos para aplicar una transformación para la carga de datos aplicando el batch_size (bs) dicho y si la data se queire revolver.
    train_loader = DataLoader(dataset=train_data, batch_size=bs, shuffle=True)
    
    mean = 0.0
    std = 0.0
    num_batches = 0

    # Se obtiene los valores exactos para usarlos en la transformación final
    for images, _ in train_loader:
        batch_mean = images.mean()
        batch_std = images.std()
        
        mean += batch_mean
        std += batch_std
        num_batches += 1

    mean /= num_batches
    std /= num_batches

    return mean, std

### Proceso de carga final

In [43]:
mean, std = getParamsTransform(bs=BATCH_SIZE)

final_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[mean], std=[std])
])

train_data = datasets.MNIST(root=BASE_DIR, train=True, download=False, transform=final_transform)
test_data = datasets.MNIST(root=BASE_DIR, train=False, download=False, transform=final_transform)

train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE, shuffle=False)

print(f"Size dataset train {len(train_data)}")
print(f"Size dataset test {len(test_data)}")

Size dataset train 60000
Size dataset test 10000


Dado la `mean` y `std` ya es posible hacer la preparación de la data y se puede hacer el dataset para `validation`, por ejemplo 90% para entrenamiento y 10% para validación del dataset de train el cual es el que contiene más datos. Pero, puede editarse la proporción para el los datos de validation editanto `FACTOR_VALIDATION_SET`.

In [44]:
train_size = int(FACTOR_VALIDATION_SET * len(train_data))
val_size = len(train_data) - train_size

train_set, val_set = random_split(train_data, [train_size, val_size])
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Al separar la data ya podemos utilizar los sets `train_loader`, `val_loader` y `test_loader`.

## Construcción del Modelo MLP

- 784 entradas (una por cada píxel de la imagen de 28x28).
- 10 salidas (una por cada clase del dígito del 0 al 9). En este caso, la neurona de salida con el valor de función de activación más alto representa la clase que el modelo está pronósticando. 

### Librerías

In [45]:
import torch.nn as nn

### Código

In [46]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.model(x)

- `nn.Flatten()`: Convierte la imagen 2D de `28x28` en un vector de `784` elementos.
- `nn.Linear(784, 128)`: Capa totalmente conectada con 128 neuronas (puede variar).
- `nn.ReLU()`: Activación no lineal común para MLP.
- `nn.Linear(128, 10)`: Capa final con 10 salidas (una por cada dígito).

## Experimentación con Distintas Configuraciones

### Librerías y constantes

In [47]:
import torch.nn as nn
import random
models = {}
mlp_instances = {}

### Código

In [48]:
class MLPConfiguration(nn.Module):
    def __init__(self, hidden_layers, activation_fn):
        """
        Parámetros:
        - hidden_layers (list[int]): número de neuronas por cada capa.
        - activation_fn (nn.Module): clase de función de activación a aplicar en cada capa oculta.

        Estructura:
        - La entrada se aplana con nn.Flatten() ya que MNIST son imágenes 28x28 (784 píxeles).
        - Cada capa oculta es una combinación de nn.Linear + activation_fn.
        - La capa final es nn.Linear que proyecta al espacio de 10 clases (dígitos 0 a 9).
        """
        super(MLPConfiguration, self).__init__()
        layers = [nn.Flatten()]  # Convierte imagen 2D a vector 1D

        input_dim = 28 * 28  # Tamaño de la entrada (por defecto para el tensor de imágenes)
        for h in hidden_layers:
            layers.append(nn.Linear(input_dim, h))     # Conectar capas
            layers.append(activation_fn())             # Activación no lineal
            input_dim = h                              # Actualiza el tamaño para la siguiente capa

        layers.append(nn.Linear(input_dim, 10))  # Capa de salida con (10 clases para los números)
        self.model = nn.Sequential(*layers)      # Juntar todas las capas

    def forward(self, x):
        return self.model(x)

In [49]:
# Modelo 1
models["mlp_1_simple"] = {
    "hidden_layers": [128],
    "activation_fn": nn.ReLU,
    "learning_rate": 0.01,
    "batch_size": 64,
    "epochs": 10
}

# Modelo 2
models["mlp_2_deep"] = {
    "hidden_layers": [256, 128],
    "activation_fn": nn.Tanh,
    "learning_rate": 0.001,
    "batch_size": 128,
    "epochs": 20
}

# Modelo 3
models["mlp_3_wide"] = {
    "hidden_layers": [512, 256],
    "activation_fn": nn.LeakyReLU,
    "learning_rate": 0.0005,
    "batch_size": 32,
    "epochs": 15
}

### Hiperparámetros utilizados

- `hidden_layers`: lista que indica cuántas capas ocultas tiene el modelo y cuántas neuronas contiene cada una.
- `activation_fn`: función de activación que se aplicará después de cada capa oculta.
- `learning_rate`: tasa de aprendizaje usada para actualizar los pesos durante el entrenamiento.
- `batch_size`: número de ejemplos procesados antes de actualizar los parámetros.
- `epochs`: número total de recorridos completos sobre el conjunto de entrenamiento.

#### Modelo 1

- Tiene una sola capa con 128 neuronas, lo que le permite aprender patrones simples y generales del conjunto de imágenes.
- Se utiliza `ReLU` como función de activación.
- Un `learning_rate` de 0.01 permite actualizaciones rápidas.
- `batch_size` de 64.
- Entrena durante 10 épocas.

#### Modelo 2

- Tiene dos capas permite al modelo componer más relacioiens con la imagen de entrada.
- La función `Tanh` comprime las salidas entre -1 y 1.
- Una tasa de aprendizaje más pequeña (`0.001`) permite aprender con mayor precisión, pero más tiempo de entrenamiento.
- El `batch_size` de 128 reduce la varianza de las actualizaciones.
- El número de épocas se incrementa a 20 para mayor capacidad de entrenamiento.

#### Modelo 3

- Posee dos capas con más neuronas.
- `LeakyReLU` es una variante de `ReLU` que evita que las neuronas “mueran” (salida siempre 0), permitiendo mantener información incluso con entradas negativas.
- El `learning_rate` bajo ayuda a que el modelo ajuste los pesos.
- Al tener un `batch_size` pequeño (32), el modelo actualiza sus pesos más frecuentemente con datos más variados.
- Entrena por 15 épocas.


#### Comparación general

| Modelo         | Capas ocultas | Activación | Learning Rate | Batch Size | Epochs |
| -------------- | ------------- | ---------- | ------------- | ---------- | ------ |
| 1              | \[128]        | ReLU       | 0.01          | 64         | 10     |
| 2              | \[256, 128]   | Tanh       | 0.001         | 128        | 20     |
| 3              | \[512, 256]   | LeakyReLU  | 0.0005        | 32         | 15     |

In [50]:
for name, config in models.items():
    model = MLPConfiguration(hidden_layers=config["hidden_layers"],
                activation_fn=config["activation_fn"])
    mlp_instances[name] = model

## Tuning de Hiperparámetros

In [51]:


search_space = {
    "hidden_layers": [[256, 128], [512, 256], [128, 64]],
    "activation_fn": [nn.Tanh],  # fija
    "learning_rate": [0.01, 0.001, 0.0005],
    "batch_size": [32, 64, 128]
}

def random_search_config(search_space, n_trials=5):
    trials = []
    for _ in range(n_trials):
        config = {
            "hidden_layers": random.choice(search_space["hidden_layers"]),
            "activation_fn": random.choice(search_space["activation_fn"]),
            "learning_rate": random.choice(search_space["learning_rate"]),
            "batch_size": random.choice(search_space["batch_size"]),
            "epochs": 20  # fijo como en el modelo original
        }
        trials.append(config)
    return trials


## Evaluación del Modelo

In [52]:
def build_mlp(input_dim, hidden_layers, output_dim, activation_fn):
    layers = []
    in_features = input_dim
    for h in hidden_layers:
        layers.append(nn.Linear(in_features, h))
        layers.append(activation_fn())
        in_features = h
    layers.append(nn.Linear(in_features, output_dim))
    return nn.Sequential(*layers)


In [53]:
def train_and_evaluate(config, train_loader, test_loader):
    model = build_mlp(784, config["hidden_layers"], 10, config["activation_fn"])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["learning_rate"])

    for epoch in range(models["mlp_2_deep"]["epochs"]):
        model.train()
        for images, labels in train_loader:
            images = images.view(-1, 28*28).to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluación
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 28*28).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

In [54]:
# 5. Realizar Random Search
def random_search(n_trials=5):
    results = []

    for _ in range(n_trials):
        config = {
            "hidden_layers": random.choice(search_space["hidden_layers"]),
            "activation_fn": search_space["activation_fn"][0],  # fijo
            "learning_rate": random.choice(search_space["learning_rate"]),
            "batch_size": random.choice(search_space["batch_size"])
        }

        train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
        test_loader = DataLoader(test_data, batch_size=1000)

        accuracy = train_and_evaluate(config, train_loader, test_loader)
        results.append((config, accuracy))
        print(f"Config: {config}, Accuracy: {accuracy:.4f}")

    return max(results, key=lambda x: x[1])


In [56]:
best_config, best_acc = random_search(n_trials=3)
print("\n🟢 Mejor configuración encontrada:")
print(best_config)
print(f"🎯 Precisión en test: {best_acc:.4f}")

Config: {'hidden_layers': [512, 256], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'learning_rate': 0.0005, 'batch_size': 32}, Accuracy: 0.9795
Config: {'hidden_layers': [256, 128], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'learning_rate': 0.001, 'batch_size': 64}, Accuracy: 0.9745
Config: {'hidden_layers': [256, 128], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'learning_rate': 0.0005, 'batch_size': 64}, Accuracy: 0.9797

🟢 Mejor configuración encontrada:
{'hidden_layers': [256, 128], 'activation_fn': <class 'torch.nn.modules.activation.Tanh'>, 'learning_rate': 0.0005, 'batch_size': 64}
🎯 Precisión en test: 0.9797


In [58]:
# Modelo 1
models["mlp_1_simple"] = {
    "hidden_layers": [128],
    "activation_fn": nn.ReLU,
    "learning_rate": 0.01,
    "batch_size": 64,
    "epochs": 10
}


# Modelo 3
models["mlp_3_wide"] = {
    "hidden_layers": [512, 256],
    "activation_fn": nn.LeakyReLU,
    "learning_rate": 0.0005,
    "batch_size": 32,
    "epochs": 15
}


for name, config in models.items():
    print(f"\n🔧 Evaluando {name}")
    train_loader = DataLoader(train_data, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test_data, batch_size=config["batch_size"])
    accuracy = train_and_evaluate(config, train_loader, test_loader)
    print(f"🎯 Precisión del modelo {name}: {accuracy:.4f}")


🔧 Evaluando mlp_1_simple
🎯 Precisión del modelo mlp_1_simple: 0.9548

🔧 Evaluando mlp_2_deep
🎯 Precisión del modelo mlp_2_deep: 0.9788

🔧 Evaluando mlp_3_wide
🎯 Precisión del modelo mlp_3_wide: 0.9837


In [ ]:
# Realizar una tabla con el ranking de las redes desde la mejor hasta la peor basada en su rendimiento.
import pandas as pd

# Datos del rendimiento de cada modelo
data = [
    {"Ranking": 1, "Modelo": "mlp_3_wide", "Precisión (%)": 98.37},
    {"Ranking": 2, "Modelo": "mlp_2_deep (Random Search)", "Precisión (%)": 97.97},
    {"Ranking": 3, "Modelo": "mlp_2_deep", "Precisión (%)": 97.88},
    {"Ranking": 4, "Modelo": "mlp_1_simple", "Precisión (%)": 95.48}
]

# Crear DataFrame
df_resultados = pd.DataFrame(data)

# Mostrar tabla ordenada
df_resultados = df_resultados.sort_values(by="Ranking").reset_index(drop=True)
print(df_resultados)


   Ranking                      Modelo  Precisión (%)
0        1                  mlp_3_wide          98.37
1        2  mlp_2_deep (Random Search)          97.97
2        3                  mlp_2_deep          97.88
3        4                mlp_1_simple          95.48


Se observa que el modelo de mlp_3_wide con sus parametros  


In [ ]:
import pandas as pd

data = {
    "Modelo": ["mlp_1_simple", "mlp_2_deep", "mlp_2_deep (Random Search)", "mlp_3_wide"],
    "hidden_layers": ["[128]", "[256, 128]", "[512, 256]", "[512, 256]"],
    "activation_fn": ["ReLU", "Tanh", "Tanh (fijo)", "LeakyReLU"],
    "learning_rate": [0.01, 0.001, 0.0005, 0.0005],
    "batch_size": [64, 128, 64, 32],
    "epochs": [10, 20, 20, 15],
    "Precisión (%)": [95.48, 97.88, 97.97, 98.37]
}

df = pd.DataFrame(data)

df


,Modelo,hidden_layers,activation_fn,learning_rate,batch_size,epochs,Precisión (%)
0,mlp_1_simple,[128],ReLU,0.0100,64,10,95.48
1,mlp_2_deep,"[256, 128]",Tanh,0.0010,128,20,97.88
2,mlp_2_deep (Random Search),"[512, 256]",Tanh (fijo),0.0005,64,20,97.97
3,mlp_3_wide,"[512, 256]",LeakyReLU,0.0005,32,15,98.37


Como podemos observar, los hiperparámetros que más influyeron en la mejora del rendimiento del modelo fueron principalmente la arquitectura de las capas ocultas, el tipo de función de activación y el learning rate. Al comparar el modelo más simple que fue mlp_1_simple con los modelos más complejos, se observa que aumentar el número de capas y neuronas permite al modelo capturar mejor patrones complejos en los datos. Además, el cambio de activación de ReLU a Tanh y LeakyReLU contribuyó a una representación más rica de la información, especialmente en combinaciones profundas o anchas. Finalmente, una tasa de aprendizaje más baja como 0.0005 en los modelos de mayor precisión permitió una convergencia más estable y precisa, evitando el sobreajuste o saltos abruptos en el descenso del gradiente. Haciendo que el modelo número 3 sea el modelo con precisión. 

## Referencias

- [Datasets PyTorch](https://docs.pytorch.org/vision/stable/datasets.html#mnist)
- [MNIST PyTorch](https://docs.pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html?highlight=mnist#torchvision.datasets.MNIST)
- [Transforming PyTorch](https://docs.pytorch.org/vision/stable/transforms.html#)
- [DataLoader PyTorch](https://docs.pytorch.org/docs/stable/data.html#)
- [Compose PyTorch](https://docs.pytorch.org/vision/stable/generated/torchvision.transforms.v2.Compose.html#torchvision.transforms.v2.Compose)
- [Torch.nn PyTorch](https://docs.pytorch.org/docs/stable/nn.html)
- [MLP PyTorch](https://docs.pytorch.org/vision/main/generated/torchvision.ops.MLP.html)
- [Sequential PyTorch](https://docs.pytorch.org/docs/stable/generated/torch.nn.Sequential.html)